In [1]:
%pip install -q --upgrade pip

Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install -q ollama pypdf chromadb numpy

Note: you may need to restart the kernel to use updated packages.


In [3]:
import ollama

In [4]:
model = "granite3.3:2b"
options = {
    "temperature": 0,
    "top-p": 0.9
}

In [5]:
def chat_assistant(question: str) -> str:
    messages = [
        {
            "role": "system",
            "content": """
            Você é um mecânico especialista em carros Hyundai HB20. 
            Sua tarefa é responder perguntas sobre a operacão, manutenção e especificação dos veículos Hyundai HB20 de forma objetiva e direta.
            Forneça respostas curtas e diretas.
            Ignore qualquer pergunta não relacionada ao veículos Hyundai HB20, respondendo de forma cordial 'Desculpe não tenho esta informacão'.
            Responda cordialmente 'Desculpe não tenho esta informacão' para perguntas que você não sabe a resposta.
            """
        },
        {
            "role": "user",
            "content": question
        } 
    ]

    resp = ollama.chat(model=model, messages=messages, options=options)
    return resp["message"]["content"]

In [6]:
print("> R:", chat_assistant("Qual a pressão em psi para carga normal dos pneus do meu HB20?"))

> R: A pressão recomendada de ar dos pneus do Hyundai HB20 é de 32 psi (PSI) para carga normal.


In [7]:
print("> R:", chat_assistant("Qual é tipo de bateria da chave inteligente do meu HB20?"))

> R: A Hyundai HB20, desde o modelo 2016 em diante, utiliza uma bateria de íon de lítio para a chave inteligente.


## RAG - Retrieval-Augmented Generation

In [8]:
import pypdf
import chromadb
import textwrap
import numpy as np

In [9]:
embedding_model = "granite-embedding:278m"

In [10]:
def extract_text_from_pdf(pdf_path: str) -> str:
    reader = pypdf.PdfReader(pdf_path)
    text_content = ""

    for page in reader.pages:
        text_content += page.extract_text()
    
    return text_content

In [11]:
documents = ["../data/manual-hb20.pdf"]

In [12]:
for document in documents:
    print(f"> Trecho do documento {document}:\n", extract_text_from_pdf(document)[:200])

> Trecho do documento ../data/manual-hb20.pdf:
 A1SO-PB2212A
(Português/Brasil)
Hyundai Motor Brasil Ltda.
Assistência 24 Horas: 0800-7703355
Impresso em 12/2022 - BAN-OP: XXXXX
HB20 Manual do Proprietário | Português
Manual do Proprietário
HB20MAN


In [17]:
import chromadb.errors


def create_collection(collection_name: str) -> chromadb.Collection:
    client = chromadb.Client()

    try:
        return client.get_collection(name=collection_name)
    except chromadb.errors.NotFoundError:
        return client.create_collection(name=collection_name)

In [18]:
def add_document_to_index(collection: chromadb.Collection, doc_name: str, doc_content: str, chunk_size=1000):
    print("> Processando o docuemnto:", doc_name)
    chunks = textwrap.wrap(doc_content, chunk_size)

    for i, chunk in enumerate(chunks):
        resp = ollama.embeddings(model=embedding_model, prompt=chunk)
        embedding = resp["embedding"]

        collection.add(
            ids=[f"{doc_name}-{i}"],
            embeddings=embedding,
            documents=[chunk],
            metadatas=[{"source": doc_name}]
        )

    print(f"> Documento {doc_name} idexado com sucesso em {len(chunks)} chunks.")


In [19]:
def ask_to_rag(question: str, collection: chromadb.Collection) -> str:
    question_embedding = ollama.embeddings(model=embedding_model, prompt=question)["embedding"]

    results = collection.query(
        query_embeddings=[question_embedding],
        n_results=2
    )

    print("> Resultados recuperados:", results)
    
    retrieved_contex = "\n".join(results["documents"][0])
    print("> Contexto recuperado:", retrieved_contex)

    messages = [
        {
            "role": "system",
            "content": """
            Você é um mecânico especialista em carros Hyundai HB20. 
            Sua tarefa é responder perguntas sobre a operacão, manutenção e especificação dos veículos Hyundai HB20 de forma objetiva e direta.
            Forneça respostas curtas e diretas.
            Ignore qualquer pergunta não relacionada ao veículos Hyundai HB20, respondendo de forma cordial 'Desculpe não tenho esta informacão'.
            Responda cordialmente 'Desculpe não tenho esta informacão' para perguntas que você não sabe a resposta.
            """
        },
        {
            "role": "documents_doc_1",
            "content": retrieved_contex
        },
        {
            "role": "user",
            "content": question
        } 
    ]

    resp = ollama.chat(model=model, messages=messages, options=options)
    return resp["message"]["content"]

In [20]:
collection = create_collection("manual_hb20")

In [21]:
for doc_name in documents:
    doc_content = extract_text_from_pdf(doc_name)
    add_document_to_index(collection, doc_name, doc_content)

> Processando o docuemnto: ../data/manual-hb20.pdf
> Documento ../data/manual-hb20.pdf idexado com sucesso em 767 chunks.


In [22]:
print("> R:", ask_to_rag(
  "Qual a pressão em psi para carga normal dos pneus do meu HB20?",
  collection))

> Resultados recuperados: {'ids': [['../data/manual-hb20.pdf-691', '../data/manual-hb20.pdf-463']], 'embeddings': None, 'documents': [['especificações (tamanhos e pressão)  podem ser encontradas na etiqueta localizada  na coluna central do lado do motorista. pressão recomendada dos pneus  frios A pressão de todos os pneus (incluindo o es- tepe) deve ser verificada com os pneus frios.  “Pneu frio” significa que o veículo está parado  há pelo menos 3 horas ou não percorreu mais  do que 1,6 km. Pneus quentes normalmente excedem a pres- são dos pneus frios em 28 a 41 kPa (4 a 6 psi).  Não libere ar dos pneus quentes para ajustar a  pressão senão eles ficarão com pressão abai- xo da indicada. A pressão recomendada para  pneus está em “Pneus e Rodas” na Seção 2.9-38 Manutenção atenÇÃo a pressão recomendada deve ser mantida  para dirigir melhor, boas manobras e mini- mo desgaste dos pneus. pressão do pneu acima ou abaixo da espe- cificada pode reduzir a vida útil do pneu,  afetar negativament

In [23]:
print("> R:", ask_to_rag(
  "Qual é tipo de bateria da chave inteligente do meu HB20?",
  collection
))

> Resultados recuperados: {'ids': [['../data/manual-hb20.pdf-197', '../data/manual-hb20.pdf-151']], 'embeddings': None, 'documents': [['do transmissor. Tipo de bateria: CR2032 Para substituir a bateria: 1. Insira uma ferramenta fina da fenda e deli - cadamente remova a tampa da chave. 2. Com uma chave de fenda remova a tampa  da bateria (A). 3. Remova a bateria usada e insira a bateria  nova (B). Certifique-se que a posição da  bateria (polaridade) esteja correta.  4. Instale a tampa da bateria e a tampa da  chave em ordem inversa à remoção. \uf06e Tipo B5-9 05 OBR2069022BR OBA0432222IN  OBR2059001BR OBR2052074BR \uf06e Tipo A \uf06e Tipo C \uf06e Tipo B uma bateria descartada de  maneira inadequada pode ser  prejudicial ao meio ambiente  e à saúde humana. descarte a  bateria de acordo com a legis- lação local pertinente. chave inteligente - smart key (se  equipado) Seu HYUNDAI usa uma chave inteligente, que  pode ser usada para travar ou destravar uma  porta (e o porta-malas/tampa tra

In [26]:
print("> R:", ask_to_rag(
  "Qual é o tempo de duração da bateria do meu HB20?",
  collection
))

> Resultados recuperados: {'ids': [['../data/manual-hb20.pdf-173', '../data/manual-hb20.pdf-683']], 'embeddings': None, 'documents': [['combustível é zerada (-- -) quando a velocidade do veículo exceder 1,5  km/h após reabastecer mais de 5 litros. NOTA a economia média de combustível não é  mostrada para cálculo mais preciso se o  veículo não rodar mais de 10 segundos ou  50 metros desde que o interruptor de igni - ção é colocado em on. OBR2042046BR4-37 04 OBR2042048BR Tempo decorrido • O tempo decorrido é o tempo total de con- dução desde o reinício do último tempo de- corrido. - Faixa de tempo (hh:mm): 00:00 ~ 99:59 • Para reiniciar manualmente, pressione o  botão OK no volante de direção por mais  de 1 segundo quando aparecer “Tempo de- corrido”. NOTA mesmo que o veículo não esteja em mo- vimento, o tempo decorrido continua en- quanto o motor estiver funcionando.4-38 Painel de instrumentos ComPutador de Bordo (tiPo B) Este computador de bordo é um sistema de  informação ao motorista

In [27]:
print("> R:", ask_to_rag(
  "Como trocar a bateria da chave inteligente do meu HB20?",
  collection
))

> Resultados recuperados: {'ids': [['../data/manual-hb20.pdf-197', '../data/manual-hb20.pdf-151']], 'embeddings': None, 'documents': [['do transmissor. Tipo de bateria: CR2032 Para substituir a bateria: 1. Insira uma ferramenta fina da fenda e deli - cadamente remova a tampa da chave. 2. Com uma chave de fenda remova a tampa  da bateria (A). 3. Remova a bateria usada e insira a bateria  nova (B). Certifique-se que a posição da  bateria (polaridade) esteja correta.  4. Instale a tampa da bateria e a tampa da  chave em ordem inversa à remoção. \uf06e Tipo B5-9 05 OBR2069022BR OBA0432222IN  OBR2059001BR OBR2052074BR \uf06e Tipo A \uf06e Tipo C \uf06e Tipo B uma bateria descartada de  maneira inadequada pode ser  prejudicial ao meio ambiente  e à saúde humana. descarte a  bateria de acordo com a legis- lação local pertinente. chave inteligente - smart key (se  equipado) Seu HYUNDAI usa uma chave inteligente, que  pode ser usada para travar ou destravar uma  porta (e o porta-malas/tampa tra

In [31]:
print("> R:", ask_to_rag(
  "Qual a pressão em psi indicada para os pneus do meu HB20?",
  collection))

> Resultados recuperados: {'ids': [['../data/manual-hb20.pdf-691', '../data/manual-hb20.pdf-620']], 'embeddings': None, 'documents': [['especificações (tamanhos e pressão)  podem ser encontradas na etiqueta localizada  na coluna central do lado do motorista. pressão recomendada dos pneus  frios A pressão de todos os pneus (incluindo o es- tepe) deve ser verificada com os pneus frios.  “Pneu frio” significa que o veículo está parado  há pelo menos 3 horas ou não percorreu mais  do que 1,6 km. Pneus quentes normalmente excedem a pres- são dos pneus frios em 28 a 41 kPa (4 a 6 psi).  Não libere ar dos pneus quentes para ajustar a  pressão senão eles ficarão com pressão abai- xo da indicada. A pressão recomendada para  pneus está em “Pneus e Rodas” na Seção 2.9-38 Manutenção atenÇÃo a pressão recomendada deve ser mantida  para dirigir melhor, boas manobras e mini- mo desgaste dos pneus. pressão do pneu acima ou abaixo da espe- cificada pode reduzir a vida útil do pneu,  afetar negativament

In [30]:
print("> R:", ask_to_rag(
  "Qual a pressão em psi para carga normal dos pneus do meu HB20?",
  collection))

> Resultados recuperados: {'ids': [['../data/manual-hb20.pdf-691', '../data/manual-hb20.pdf-463']], 'embeddings': None, 'documents': [['especificações (tamanhos e pressão)  podem ser encontradas na etiqueta localizada  na coluna central do lado do motorista. pressão recomendada dos pneus  frios A pressão de todos os pneus (incluindo o es- tepe) deve ser verificada com os pneus frios.  “Pneu frio” significa que o veículo está parado  há pelo menos 3 horas ou não percorreu mais  do que 1,6 km. Pneus quentes normalmente excedem a pres- são dos pneus frios em 28 a 41 kPa (4 a 6 psi).  Não libere ar dos pneus quentes para ajustar a  pressão senão eles ficarão com pressão abai- xo da indicada. A pressão recomendada para  pneus está em “Pneus e Rodas” na Seção 2.9-38 Manutenção atenÇÃo a pressão recomendada deve ser mantida  para dirigir melhor, boas manobras e mini- mo desgaste dos pneus. pressão do pneu acima ou abaixo da espe- cificada pode reduzir a vida útil do pneu,  afetar negativament

In [32]:
print("> R:", ask_to_rag(
  "Qual é tipo de bateria da chave inteligente do meu HB20?",
  collection
))

> Resultados recuperados: {'ids': [['../data/manual-hb20.pdf-197', '../data/manual-hb20.pdf-151']], 'embeddings': None, 'documents': [['do transmissor. Tipo de bateria: CR2032 Para substituir a bateria: 1. Insira uma ferramenta fina da fenda e deli - cadamente remova a tampa da chave. 2. Com uma chave de fenda remova a tampa  da bateria (A). 3. Remova a bateria usada e insira a bateria  nova (B). Certifique-se que a posição da  bateria (polaridade) esteja correta.  4. Instale a tampa da bateria e a tampa da  chave em ordem inversa à remoção. \uf06e Tipo B5-9 05 OBR2069022BR OBA0432222IN  OBR2059001BR OBR2052074BR \uf06e Tipo A \uf06e Tipo C \uf06e Tipo B uma bateria descartada de  maneira inadequada pode ser  prejudicial ao meio ambiente  e à saúde humana. descarte a  bateria de acordo com a legis- lação local pertinente. chave inteligente - smart key (se  equipado) Seu HYUNDAI usa uma chave inteligente, que  pode ser usada para travar ou destravar uma  porta (e o porta-malas/tampa tra